In [1]:
import numpy as np
import pandas as pd
import pprint, pickle
#from pycaret.utils import version
#version()
import pandas as pd
import numpy as np
#from itertools import chain
from sklearn.ensemble import IsolationForest
from sklearn.impute import KNNImputer


In [2]:
name_list=['chathura','hasitha','yohan','ravindu','geemanthi','jayasanka','derek','nirmana','dasith','bimsara']

In [3]:
c=['side','front']
#direction='right'
#vid_no=3

In [4]:
#name_list=['ishin']

In [5]:
for cam in c:
    if cam == 'front':
        d=['front','back']
    elif cam == 'side':
        d=['right','left']

    for direction in d:
        for name in name_list:
            for vid_no in range(1, 4):
                print(name)
                s_file_S = open('parameters/Static/{}/{}/{}_{}_{}.pkl'.format(cam,direction,name,direction,vid_no), 'rb')
                s_file_D = open('parameters/Dynamic/{}/{}/{}_{}_{}.pkl'.format(cam,direction,name,direction,vid_no), 'rb')
                sT_S = pickle.load(s_file_S)
                sT_D = pickle.load(s_file_D)
                s_file_S.close()
                s_file_D.close()
                #print(sT_S)
                #print(sT_D)

                merged_df=[]
                gait_merged=[]
                data=[]
                data_missing=[]
                imputer=[]
                data_imputed=[]
                data_NaN_handled=[]
                model=[]
                outliers=[]
                data_outliered=[]
                time=[]
                output=[]

                # merge the two dataframes based on the common 'time' column
                merged_df = pd.merge(sT_S, sT_D, on='time', how='outer')

                # print the merged dataframe
                merged_df = merged_df.sort_values(by='time')
                merged_df = merged_df.reset_index(drop=True)
                count_col = merged_df.pop("count")
                merged_df.insert(0, "count", count_col)

                merged_df

                ## drop count & time columns
                gait_merged=merged_df.iloc[:, 2:]
                gait_merged
                print('gait_merged.shape',gait_merged.shape)

                #handle missing NAN values
                data = gait_merged


                # Loop over each column in the dataset
                for col in data.columns:
                    # Calculate mean of non-NaN values in the current column
                    col_mean = data[col][~data[col].isna()].mean()

                    # Replace NaN values in the current column with the calculated mean
                    data[col] = data[col].fillna(col_mean)

                # Apply mean imputation to handle missing values

                # Convert the imputed data back to a Pandas DataFrame
                data_NaN_handled = pd.DataFrame(data, columns=data.columns)

                # Save the imputed data to a CSV file
                print('data_NaN_handled.shape',data_NaN_handled.shape)
                data_NaN_handled

                #Handling Outliers

                # Define the Isolation Forest model with a contamination parameter of 0.05 (i.e., 5% of data points are expected to be outliers)
                model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.05, random_state=42)

                # Fit the model to the data and predict the outliers
                model.fit(data_NaN_handled)
                outliers = model.predict(data_NaN_handled)
                print(outliers)

                # Remove the outliers from the dataset
                data_outliered = data_NaN_handled[outliers == 1]

                # Print the number of outliers detected and the number of data points remaining after outlier removal
                print(f'Number of outliers detected: {len(data_NaN_handled[outliers == -1])}')
                print(f'Number of data points remaining after outlier removal: {len(data_outliered)}')

                time = merged_df[['time']]

                # merge time dataframe with data_outliered dataframe
                data_outliered.insert(0, 'time', merged_df['time'])
                data_outliered = data_outliered.reset_index(drop=True)


                # show data_outliered dataframe with the corresponding time column
                print('data_outliered.shape',data_outliered.shape)
                print(data_outliered)

                output = open('feature_vector/{}/{}/FV_{}_{}.pkl'.format(cam,direction,name,vid_no), 'wb')
                pickle.dump(data_outliered, output)
                output.close()
                print(cam,direction,name,vid_no)
                print("____________________________________________________")
    


chathura
gait_merged.shape (6, 56)
data_NaN_handled.shape (6, 56)


/home/cbk/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


[-1  1  1  1  1  1]
Number of outliers detected: 1
Number of data points remaining after outlier removal: 5
data_outliered.shape (5, 57)
       time  lower_body_left_avg  lower_body_right_avg  upper_arm_left_avg  \
0  1.900000             1.349872              1.338067            0.524864   
1  2.433333             1.250626              1.351073            0.519489   
2  3.000000             1.311683              1.296826            0.535490   
3  3.566667             1.270731              1.361292            0.537342   
4  4.133333             1.373674              1.308752            0.537279   

   upper_arm_right_avg  lower_arm_left_avg  lower_arm_right_avg  arm_left_avg  \
0             0.547882            0.412736             0.469015      0.287622   
1             0.560115            0.396591             0.471024      0.282546   
2             0.568994            0.408922             0.448197      0.285756   
3             0.581611            0.409023             0.467527      0

/home/cbk/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


[-1  1  1  1  1  1]
Number of outliers detected: 1
Number of data points remaining after outlier removal: 5
data_outliered.shape (5, 57)
       time  lower_body_left_avg  lower_body_right_avg  upper_arm_left_avg  \
0  2.066667             1.280808              1.376064            0.513282   
1  2.600000             1.310650              1.297259            0.527299   
2  3.166667             1.261249              1.358281            0.528309   
3  3.666667             1.318029              1.294130            0.536622   
4  4.300000             1.287707              1.379193            0.542975   

   upper_arm_right_avg  lower_arm_left_avg  lower_arm_right_avg  arm_left_avg  \
0             0.555741            0.415286             0.469488      0.282036   
1             0.557166            0.390680             0.462868      0.277924   
2             0.577043            0.401006             0.465939      0.285066   
3             0.573442            0.401198             0.440652      0

...

/home/cbk/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/cbk/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


[1 1]
Number of outliers detected: 0
Number of data points remaining after outlier removal: 2
data_outliered.shape (2, 49)
       time  df_lower_body_length_avg  df_shoulder_length_avg  \
0  1.000000                  1.342393                0.359179   
1  2.133333                  1.341441                0.346554   

   df_hip_size_avg  df_upper_arm_length_left_avg  \
0         0.195198                      0.523528   
1         0.192767                      0.511943   

   df_upper_arm_length_right_avg  df_lower_arm_length_left_avg  \
0                       0.517666                      0.415610   
1                       0.520059                      0.422142   

   df_lower_arm_length_right_avg  df_arm_length_left_avg  \
0                       0.429810                1.050326   
1                       0.426507                1.052284   

   df_arm_length_right_avg  ...       fll        po        sw  visi_stl  \
0                 1.067108  ...  0.255755  6.370215  0.153747  0.9864

/home/cbk/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
/home/cbk/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


[1 1]
Number of outliers detected: 0
Number of data points remaining after outlier removal: 2
data_outliered.shape (2, 49)
       time  df_lower_body_length_avg  df_shoulder_length_avg  \
0  1.166667                  1.286199                0.305681   
1  1.933333                  1.340233                0.360805   

   df_hip_size_avg  df_upper_arm_length_left_avg  \
0         0.167249                      0.535038   
1         0.204386                      0.523064   

   df_upper_arm_length_right_avg  df_lower_arm_length_left_avg  \
0                        0.53385                      0.360147   
1                        0.50872                      0.379046   

   df_lower_arm_length_right_avg  df_arm_length_left_avg  \
0                       0.439029                1.022239   
1                       0.430245                1.016519   

   df_arm_length_right_avg  ...       fll         po        sw  visi_stl  \
0                 1.104895  ...  0.173787  42.675019  0.125828  0.96

/home/cbk/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(
